## Agentic AI Framework Project

#### This notebook will showcase the follwoing:

* Agent workflow
* Use of tools to call functions
* Agent collaboration via Tools and Handoffs

In [1]:
import asyncio
from dotenv import load_dotenv
from agents import Agent, trace, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

### Step1: Agent Workflow

In [3]:
instructions1 = """You are a sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious \
and professional
"""

instructions2 = """You are a humorous, engaging sales agent working for InsightStat, which is a company offering data science solutions. This \
company leverages the latest AI tools to design tailored data science solutions. Your job is to write witty, engaging cold emails will likely get you \
a response"""

instructions3 = """You are a busy ales agent working for InsightStat, which is a company offering data science solutions. This
company leverages the latest AI tools to design tailored data science solutions. Your job is to write serious cold emails that are concise
and to the point"""

In [4]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [5]:
result = Runner.run_streamed(sales_agent1, input= "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush=True)

Subject: Unlock the Power of Data Science with InsightStat

Dear [Recipient's Name],

I hope this message finds you well.

I’m [Your Name], a sales consultant at InsightStat, and I wanted to reach out to discuss how our specialized data science solutions can help your organization leverage the power of data to drive informed decisions and achieve your strategic goals.

At InsightStat, we recognize that every business has unique challenges. Our team utilizes the latest AI tools and methodologies to create tailored solutions that can enable you to:

- Gain actionable insights from your data
- Optimize processes to improve efficiency
- Enhance customer experiences through predictive analytics

We have successfully partnered with companies in various sectors, helping them transform their data into a valuable asset. I would love to discuss how we can support your specific goals and challenges.

Would you be available for a quick call next week? Please let me know a time that works for you, 

In [6]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output, "\n\n")

Subject: Unlock New Insights with Tailored Data Science Solutions

Dear [Recipient's Name],

I hope this email finds you well. My name is [Your Name], and I’m reaching out on behalf of InsightStat, where we specialize in providing cutting-edge data science solutions tailored to meet the unique needs of our clients.

In today's data-driven landscape, harnessing the power of information is crucial for making informed decisions and gaining a competitive edge. At InsightStat, we leverage advanced AI tools and a team of experienced data scientists to help organizations like yours uncover insights, optimize processes, and drive growth.

Whether you are looking to improve predictive analytics, automate reporting, or unlock hidden patterns in your data, we can design a solution that aligns perfectly with your business objectives.

I would love to arrange a brief call to discuss how InsightStat can support your data initiatives and answer any questions you may have. What does your schedule look

In [7]:
sales_picker = Agent(
    name = "sales_picker",
    instructions = "You have to pick the best cold sales email from the given oprions \
        Imagine you are a customer who will respond to one of the emails that you liked the best \
        Do not give any explanation. Just return the best email",
        model= "gpt-4o-mini" 
)

In [8]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

emails = "Cold sales emails:\n\n" + "\n\nEmail: \n\n".join(outputs)

best = await Runner.run(sales_picker, emails)

print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Let’s Make Your Data Dance! 💃📊

Hi [Recipient's Name],

I hope this email finds you swimming in data and not drowning in spreadsheets! 🏊‍♂️ If you’re like most companies these days, your data is both your best friend and that friend who always wants to talk about their cat's dietary preferences—endlessly fascinating but possibly overwhelming.

At InsightStat, we believe data should work for you, not the other way around. Think of us as your personal trainers for data science—turning those heavy datasets into lean, mean insights machines! 💪✨

We specialize in tailoring AI-powered solutions that fit your unique business needs. Whether you want to predict customer trends, optimize operations, or just figure out why your last campaign fell flatter than a pancake at a bad brunch, we’ve got your back.

Fancy a chat over a virtual coffee? I promise to bring the insights and leave the data puns at the door (well, mostly). Let’s turn your data dilemmas into data delig

### Use of Tools

In [9]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = 'gpt-4o-mini'
)

In [10]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for InsightStat, which is a company offering data science solutions. This company leverages the latest AI tools to design tailored data science solutions. Your job is to write cold emails that are serious and professional\n', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [11]:
@function_tool
def send_email(body:str):
    """ Send out an email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ['SENDGRID_API_KEY'])
    from_email = Email("kausthab.phukan@gmail.com")
    to_email = To("kausthab.phukan@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content)
    sg.send(mail)
    return {"status":"success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD926B60>, strict_json_schema=True, is_enabled=True)

Converting Agent into a tool

In [13]:
tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= "Write a cold sales email")

tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD983740>, strict_json_schema=True, is_enabled=True)

If we can do it for 1, we can do it for all the others

In [14]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name= "sales_agent1", tool_description= description)
tool2 = sales_agent2.as_tool(tool_name= "sales_agent2", tool_description= description)
tool3 = sales_agent3.as_tool(tool_name= "sales_agent3", tool_description= description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD982340>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD9720C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

### Creating the Sales Manager that will select the best email to send

In [16]:
instructions = """You are Sales Manager at Insight Stat and responsible for looking over all the communications your sales agents send out to their prospects. \
    It is also your responsibility to select the best cold sales email using the sales agent tool. And this is how you should go about doing it:

    1.  Use the sales_agent1, sales_agent2, and sales_agent3 tools to generate three different email drafts. Call each tool once to get one draft from each. \
    Do not proceed until all the drafts have been prepared and are ready.


    2. Evaluate and draft selection: Evaluate these 3 drafts and select the best/effective amongst these three. You do not need to give any explanation.

    3.  Use the send_email tool to send the selected best email (only the best email) to the prospect 

    Critical rule:

    - You must use the  sales_agent1, sales_agent2, and sales_agent3 tools to write the emails, never write the emails yourself

    - You will use the send_email tool to send only one email to the prospect, and never send more than one email
"""

sales_manager = Agent(name= "sales_manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "send a cold email to a prospect addressing 'Dear CEO' "

with trace("sales manager"):
    result = await Runner.run(sales_manager, message, max_turns=30)

## Handoffs

In [17]:
subject_instruction = "You are an expert in writing cold email subjects \
    You will be given the body of a cold email and you have to come up with an engaging subject \
    Such that it will generate a response"

html_instruction = "You are an expert in converting text mail body to HTML email body \
    You are given an email with text body that might also contain markdown elements \
    You have to convert it into an HTML layout with simple, clear and compelling layout and design"

subject_writer = Agent(name = "subject_writer", instructions=subject_instruction, model = "gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description= "write an engaging subject line for a cold sales email")


html_converter = Agent(name= "html_converter", instructions= html_instruction, model = 'gpt-4o-mini')
html_tool = html_converter.as_tool(tool_name= "html_converter", tool_description= "convert a plain or text body email into a html email body")

In [18]:
@function_tool
def send_html_email(subject:str, html_body:str) -> Dict[str,str]:
    """ Send out an email with given subject and HTML body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ['SENDGRID_API_KEY'])
    from_email = Email("kausthab.phukan@gmail.com")
    to_email = To("kausthab.phukan@gmail.com")
    content = Content("text/html",html_body)
    mail = Mail(from_email, to_email, subject, content)
    sg.send(mail)
    return {"status":"success"}

In [19]:
tools = [subject_tool, html_tool, send_html_email]

In [20]:
tools

[FunctionTool(name='subject_writer', description='write an engaging subject line for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CF40DEE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='convert a plain or text body email into a html email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CF40F740>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with given subj

In [21]:

instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(name = "Email Manager",
            instructions=instructions,
            tools = tools,
            model= "gpt-4o-mini",
            handoff_description= "Convert an email into HTML and send it"
                )

In [22]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD982340>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001A0CD9720C0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [23]:
sales_manager_instructions =  """You are Sales Manager at Insight Stat and responsible for looking over all the communications your sales agents send out to their prospects. \
    It is also your responsibility to select the best cold sales email using the sales agent tool. And this is how you should go about doing it:

    1.  Use the sales_agent1, sales_agent2, and sales_agent3 tools to generate three different email drafts. Call each tool once to get one draft from each. \
    Do not proceed until all the drafts have been prepared and are ready.


    2. Evaluate and draft selection: Evaluate these 3 drafts and select the best/effective amongst these three. You do not need to give any explanation.
    You can call the tools multiple times if you are not satisfied with the drafts from the first try

    3. For Handoff and Sending: send only the best winning Email to the "Email Manager" agent. The Email Manager agent will take care of formatting and sending

    Critical rule:

    - You must use the  sales_agent1, sales_agent2, and sales_agent3 tools to write the emails, never write the emails yourself

    - You will Hand Off one and only one email to the Email Manager agent - never more than one
"""

sales_manager = Agent(name = "Sales Manager",
        instructions= sales_manager_instructions,
        tools = tools,
        handoffs=handoffs,
        model = "gpt-4o-mini")

message = "Send a cold sales email to 'Dear CEO' from Kevin"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message, max_turns=30)